In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import math

In [53]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return (self.data.shape[1] - self.sequence_length) * len(self.data)

    def __getitem__(self, index):
        account_idx = index // (self.data.shape[1] - self.sequence_length)
        seq_idx = index % (self.data.shape[1] - self.sequence_length)
        x = self.data[account_idx, seq_idx:seq_idx + self.sequence_length]
        y = self.data[account_idx, seq_idx + self.sequence_length]
        return torch.tensor(x, dtype=torch.float32).unsqueeze(-1), torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

In [54]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [55]:
data = pd.read_csv('Data.csv')
data = data.drop(columns=['Account No'])  # Drop the account number column
data = data.values  # Convert to numpy array

In [56]:
scaler = MinMaxScaler()
data = scaler.fit_transform(data.T).T  # Normalize each time series

In [57]:
sequence_length = 5
dataset = TimeSeriesDataset(data, sequence_length)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [58]:
input_size = 1
hidden_size = 64
num_layers = 2
output_size = 1
num_epochs = 30
learning_rate = 0.001

In [59]:
model = RNN(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [60]:
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for sequences, targets in dataloader:
        sequences = sequences.view(-1, sequence_length, input_size)
        targets = targets.view(-1, output_size)  # Ensure target shape is (batch_size, output_size)
        outputs = model(sequences)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    if (epoch+1) % 1 == 0:
        rmse = math.sqrt(epoch_loss / len(dataloader))
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, RMSE: {rmse:.4f}')

print("Training complete.")

Epoch [1/30], Loss: 0.0047, RMSE: 0.2299
Epoch [2/30], Loss: 0.1220, RMSE: 0.2187
Epoch [3/30], Loss: 0.0104, RMSE: 0.2143
Epoch [4/30], Loss: 0.0116, RMSE: 0.2126
Epoch [5/30], Loss: 0.0550, RMSE: 0.2120
Epoch [6/30], Loss: 0.0057, RMSE: 0.2115
Epoch [7/30], Loss: 0.0504, RMSE: 0.2115
Epoch [8/30], Loss: 0.1243, RMSE: 0.2107
Epoch [9/30], Loss: 0.0018, RMSE: 0.2107
Epoch [10/30], Loss: 0.0218, RMSE: 0.2104
Epoch [11/30], Loss: 0.1268, RMSE: 0.2104
Epoch [12/30], Loss: 0.0142, RMSE: 0.2099
Epoch [13/30], Loss: 0.0593, RMSE: 0.2099
Epoch [14/30], Loss: 0.0407, RMSE: 0.2101
Epoch [15/30], Loss: 0.0164, RMSE: 0.2096
Epoch [16/30], Loss: 0.0189, RMSE: 0.2097
Epoch [17/30], Loss: 0.0008, RMSE: 0.2093
Epoch [18/30], Loss: 0.0511, RMSE: 0.2092
Epoch [19/30], Loss: 0.0034, RMSE: 0.2092
Epoch [20/30], Loss: 0.3216, RMSE: 0.2090
Epoch [21/30], Loss: 0.0474, RMSE: 0.2088
Epoch [22/30], Loss: 0.0118, RMSE: 0.2088
Epoch [23/30], Loss: 0.1259, RMSE: 0.2085
Epoch [24/30], Loss: 0.0030, RMSE: 0.2089
E

In [61]:
torch.save(model.state_dict(), 'rnn_model.pth')

In [2]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot